# RO System Simulation Template

This notebook template is used by the simulate_ro_system tool to run WaterTAP simulations.
It accepts parameters via papermill and returns performance results.

In [1]:
# Parameters cell - will be replaced by papermill
project_root = "/path/to/project"  # Will be replaced by papermill
configuration = {}
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
membrane_properties = None
optimize_pumps = True  # Changed default to True to match simulate_ro.py
# Solver paths (optional, passed by simulate_ro.py)
idaes_bin_dir = None
pyomo_lib_path = None

In [2]:
# Parameters
project_root = "C:\\Users\\hvksh\\mcp-servers\\ro-design-mcp"
configuration = {
    "feed_flow_m3h": 100,
    "n_stages": 2,
    "stage_count": 2,
    "stages": [
        {
            "stage_number": 1,
            "n_vessels": 10,
            "membrane_area_m2": 2601.2,
            "stage_recovery": 0.5,
            "feed_flow_m3h": 100,
        },
        {
            "stage_number": 2,
            "n_vessels": 5,
            "membrane_area_m2": 1300.6,
            "stage_recovery": 0.5,
            "feed_flow_m3h": 50,
        },
    ],
    "total_recovery": 0.75,
    "array_notation": "10:5",
}
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
optimize_pumps = True


In [3]:
# Environment setup - configure solver paths
import os
import platform

# Add solver paths to environment if provided
if idaes_bin_dir and os.path.exists(idaes_bin_dir):
    if 'PATH' in os.environ:
        os.environ['PATH'] = f"{idaes_bin_dir};{os.environ['PATH']}"
    else:
        os.environ['PATH'] = idaes_bin_dir
    print(f"Added IDAES binary directory to PATH: {idaes_bin_dir}")

if pyomo_lib_path and os.path.exists(pyomo_lib_path):
    if 'PATH' in os.environ:
        os.environ['PATH'] = f"{pyomo_lib_path};{os.environ['PATH']}"
    else:
        os.environ['PATH'] = pyomo_lib_path
    print(f"Added Pyomo lib path to PATH: {pyomo_lib_path}")

# Verify solver availability
try:
    from pyomo.environ import SolverFactory
    solver = SolverFactory('ipopt')
    if solver.available():
        print(f"ipopt solver is available at: {solver.executable()}")
    else:
        print("WARNING: ipopt solver not available!")
        # Try to find it manually
        ipopt_exe = "ipopt.exe" if platform.system() == "Windows" else "ipopt"
        for path_dir in os.environ.get('PATH', '').split(';' if platform.system() == 'Windows' else ':'):
            ipopt_path = os.path.join(path_dir, ipopt_exe)
            if os.path.exists(ipopt_path):
                print(f"Found ipopt at: {ipopt_path}")
                solver.set_executable(ipopt_path)
                if solver.available():
                    print("Successfully configured ipopt solver!")
                break
except Exception as e:
    print(f"Error checking solver: {e}")

ipopt solver is available at: C:\Users\hvksh\AppData\Local\idaes\bin\ipopt.exe


In [4]:
# Import required libraries
import numpy as np
import pandas as pd
from pyomo.environ import *
from pyomo.network import Arc
import sys
import os

# Add parent directory to path for utils imports using project_root parameter
sys.path.insert(0, project_root)

# Import our elegant initialization utilities
from utils.ro_initialization import (
    calculate_required_pressure,
    initialize_pump_with_pressure,
    initialize_ro_unit_elegant,
    initialize_multistage_ro_elegant
)

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType
)
from watertap.unit_models.pressure_changer import Pump
from watertap.property_models.multicomp_aq_sol_prop_pack import MCASParameterBlock
import watertap.property_models.seawater_prop_pack as props_sw

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.util import DiagnosticsToolbox
from idaes.models.unit_models import Feed, Product

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(level=logging.INFO)

# Results storage
results = {}

## Build WaterTAP Model

In [5]:
def build_ro_model_simple(config_data, feed_salinity_ppm, feed_temperature_c, membrane_type):
    """
    Build simplified WaterTAP RO model using fixed pressure drops and SKK transport model.
    This avoids FBBT issues with calculated pressure change.
    """
    # Create concrete model
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Import TransportModel
    from watertap.core.membrane_channel_base import TransportModel
    
    # Property package - using seawater
    m.fs.properties = props_sw.SeawaterParameterBlock()
    
    # Feed conditions
    feed_flow_m3_s = config_data['feed_flow_m3h'] / 3600  # Convert to m³/s
    feed_mass_frac = feed_salinity_ppm / 1e6  # Convert ppm to mass fraction
    
    # Create feed unit
    m.fs.feed = Feed(property_package=m.fs.properties)
    
    # Build stages
    n_stages = config_data['stage_count']
    
    # First, create all RO stages and pumps using setattr for proper parent assignment
    for i in range(1, n_stages + 1):
        # Create feed pump for stage using setattr
        setattr(m.fs, f"pump{i}", Pump(property_package=m.fs.properties))
        
        # Create RO stage with SIMPLIFIED configuration and SKK model
        setattr(m.fs, f"ro_stage{i}", ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            concentration_polarization_type=ConcentrationPolarizationType.none,
            mass_transfer_coefficient=MassTransferCoefficient.none,
            pressure_change_type=PressureChangeType.fixed_per_stage,
            transport_model=TransportModel.SKK  # Use SKK model
        ))
        
        # Create permeate product for each stage
        setattr(m.fs, f"stage_product{i}", Product(property_package=m.fs.properties))
    
    # Create final concentrate product
    m.fs.concentrate_product = Product(property_package=m.fs.properties)
    
    # Connect feed to first pump
    m.fs.feed_to_pump1 = Arc(
        source=m.fs.feed.outlet,
        destination=m.fs.pump1.inlet
    )
    
    # Connect first pump to first RO
    m.fs.pump1_to_ro1 = Arc(
        source=m.fs.pump1.outlet,
        destination=m.fs.ro_stage1.inlet
    )
    
    # Connect permeate from first RO to product
    m.fs.ro1_perm_to_prod = Arc(
        source=m.fs.ro_stage1.permeate,
        destination=m.fs.stage_product1.inlet
    )
    
    # Connect stages
    if n_stages > 1:
        for i in range(1, n_stages):
            # Connect concentrate of stage i to pump i+1
            setattr(
                m.fs, f"stage{i}_to_pump{i+1}",
                Arc(
                    source=getattr(m.fs, f"ro_stage{i}").retentate,
                    destination=getattr(m.fs, f"pump{i+1}").inlet
                )
            )
            # Connect pump i+1 to stage i+1
            setattr(
                m.fs, f"pump{i+1}_to_stage{i+1}",
                Arc(
                    source=getattr(m.fs, f"pump{i+1}").outlet,
                    destination=getattr(m.fs, f"ro_stage{i+1}").inlet
                )
            )
            # Connect permeate to product
            setattr(
                m.fs, f"ro{i+1}_perm_to_prod{i+1}",
                Arc(
                    source=getattr(m.fs, f"ro_stage{i+1}").permeate,
                    destination=getattr(m.fs, f"stage_product{i+1}").inlet
                )
            )
    
    # Connect final concentrate to product
    final_stage = n_stages
    m.fs.final_conc_arc = Arc(
        source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
        destination=m.fs.concentrate_product.inlet
    )
    
    # Apply arcs to expand the network
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    # NOW set membrane properties after model structure is built
    for i in range(1, n_stages + 1):
        stage_data = config_data['stages'][i-1]
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Set reflection coefficient for SKK model
        if membrane_type == "seawater":
            ro.reflect_coeff.fix(0.98)  # Higher rejection for seawater
        else:  # brackish
            ro.reflect_coeff.fix(0.95)  # Standard rejection for brackish
        
        # Default membrane properties based on type
        if membrane_type == "seawater":
            ro.A_comp.fix(1.5e-12)  # m/s/Pa
            ro.B_comp[0, 'TDS'].fix(1.0e-8)  # m/s
        else:  # brackish
            ro.A_comp.fix(4.2e-12)  # m/s/Pa
            ro.B_comp[0, 'TDS'].fix(3.5e-8)  # m/s
        
        # Fix permeate pressure and pressure drop
        ro.permeate.pressure.fix(101325)  # 1 atm
        ro.deltaP.fix(-0.5e5)  # -0.5 bar pressure drop
        
        # Set membrane area
        ro.area.fix(stage_data['membrane_area_m2'])
    
    # Set feed conditions
    m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(
        feed_flow_m3_s * 1000 * (1 - feed_mass_frac)
    )
    m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'].fix(
        feed_flow_m3_s * 1000 * feed_mass_frac
    )
    m.fs.feed.outlet.temperature.fix(273.15 + feed_temperature_c)
    m.fs.feed.outlet.pressure.fix(101325)  # 1 atm
    
    # Set pump efficiencies
    for i in range(1, n_stages + 1):
        getattr(m.fs, f"pump{i}").efficiency_pump.fix(0.8)
    
    # Set scaling
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
    calculate_scaling_factors(m)
    
    return m

def initialize_and_solve_elegant(m, config_data, optimize_pumps=False):
    """
    Initialize and solve the RO model using elegant initialization approach.
    
    When optimize_pumps=False: Just initialize and solve with fixed pumps
    When optimize_pumps=True: Unfix pumps and add recovery constraints to match targets exactly
    """
    # Get solver (no parameters to get_solver)
    solver = get_solver()
    
    # Check initial degrees of freedom
    print(f"Initial degrees of freedom: {degrees_of_freedom(m)}")
    
    # Use elegant initialization approach (pumps are fixed during this step)
    try:
        initialize_multistage_ro_elegant(m, config_data, verbose=True)
        print("\nInitialization successful using elegant approach!")
    except Exception as e:
        print(f"\nInitialization error: {str(e)}")
        raise
    
    if not optimize_pumps:
        # Simple solve with fixed pumps (no recovery constraints)
        print("\nSolving with fixed pump pressures...")
        results = solver.solve(m, tee=True, options={'linear_solver': 'ma27'})
        
        if results.solver.termination_condition == TerminationCondition.optimal:
            print("\nSolution found with fixed pumps!")
            # Report actual recoveries achieved
            print("\nActual recoveries achieved:")
            for i in range(1, config_data['stage_count'] + 1):
                ro = getattr(m.fs, f"ro_stage{i}")
                actual_recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
                print(f"  Stage {i}: {actual_recovery:.1%}")
        
        return results
    
    # If optimize_pumps=True, proceed with optimization
    print("\nOptimization requested - unfixing pump pressures...")
    
    # First solve with fixed pumps to verify feasibility
    print("\nVerifying initial solution...")
    results = solver.solve(m, tee=False, options={'linear_solver': 'ma27'})
    if results.solver.termination_condition != TerminationCondition.optimal:
        print(f"Initial solve failed: {results.solver.termination_condition}")
        raise RuntimeError("Initial solve failed after initialization")
    
    # Now unfix pump pressures for optimization
    for i in range(1, config_data['stage_count'] + 1):
        pump = getattr(m.fs, f"pump{i}")
        current_pressure = value(pump.outlet.pressure[0])
        
        # Unfix and set bounds
        pump.outlet.pressure[0].unfix()
        pump.outlet.pressure[0].setlb(5e5)   # 5 bar minimum
        pump.outlet.pressure[0].setub(80e5)  # 80 bar maximum
        
        print(f"  Stage {i}: Unfixed at {current_pressure/1e5:.1f} bar, bounds: [5, 80] bar")
    
    # Add recovery constraints to match configuration targets
    print("\nAdding recovery constraints...")
    from pyomo.environ import Constraint
    
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        target_recovery = stage_data.get('stage_recovery', 0.5)
        
        # Add constraint for target recovery
        setattr(m.fs, f"recovery_constraint_stage{i}",
            Constraint(
                expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] == target_recovery
            )
        )
        print(f"  Stage {i}: Target recovery = {target_recovery:.1%}")
    
    # Add minimum net driving pressure constraints for SKK model
    print("\nAdding minimum net driving pressure constraints (SKK model)...")
    
    # Define minimum net driving pressure (e.g., 2 bar = 2e5 Pa)
    MIN_NET_DRIVING_PRESSURE = 2e5  # Pa
    
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # For SKK model, ensure positive net driving pressure
        # Net driving pressure = (P_feed - P_perm) - σ(π_feed - π_perm) > 0
        
        # Get reflection coefficient
        sigma = ro.reflect_coeff
        
        # Pressure difference
        p_feed = ro.inlet.pressure[0]
        p_perm = ro.permeate.pressure[0]
        
        # Osmotic pressure difference
        # For seawater property package, use pressure_osm
        pi_feed = ro.feed_side.properties_in[0].pressure_osm_phase[('Liq',)]
        pi_perm = ro.mixed_permeate[0].pressure_osm_phase[('Liq',)]
        
        # Add constraint for positive net driving pressure
        setattr(m.fs, f"net_driving_pressure_stage{i}",
            Constraint(
                expr=(p_feed - p_perm) - sigma * (pi_feed - pi_perm) >= MIN_NET_DRIVING_PRESSURE
            )
        )
        
        print(f"  Stage {i}: Added minimum net driving pressure constraint (>= {MIN_NET_DRIVING_PRESSURE/1e5:.1f} bar)")
    
    # Check DOF after adding constraints
    print(f"\nDegrees of freedom after all constraints: {degrees_of_freedom(m)}")
    
    # Solve with recovery constraints and unfixed pumps
    print("\nSolving model with recovery and pressure constraints...")
    results = solver.solve(m, tee=True, options={'linear_solver': 'ma27'})
    
    if results.solver.termination_condition == TerminationCondition.optimal:
        # Verify recoveries match targets
        print("\nVerifying recovery targets:")
        max_error = 0
        for i in range(1, config_data['stage_count'] + 1):
            ro = getattr(m.fs, f"ro_stage{i}")
            actual_recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
            target_recovery = config_data['stages'][i-1].get('stage_recovery', 0.5)
            error = abs(actual_recovery - target_recovery)
            max_error = max(max_error, error)
            print(f"  Stage {i}: Target = {target_recovery:.4f}, Actual = {actual_recovery:.4f}, Error = {error:.2e}")
        
        if max_error > 1e-4:
            print(f"\nWARNING: Maximum recovery error ({max_error:.2e}) exceeds tolerance!")
        else:
            print(f"\nSUCCESS: All recoveries match targets within tolerance!")
        
        print("\nOptimized conditions (SKK model):")
        for i in range(1, config_data['stage_count'] + 1):
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            
            # Get pressures
            feed_pressure = value(pump.outlet.pressure[0]) / 1e5
            perm_pressure = value(ro.permeate.pressure[0]) / 1e5
            
            # Get osmotic pressures
            feed_osm = value(ro.feed_side.properties_in[0].pressure_osm_phase[('Liq',)]) / 1e5
            perm_osm = value(ro.mixed_permeate[0].pressure_osm_phase[('Liq',)]) / 1e5
            
            # Get reflection coefficient
            reflect_coeff = value(ro.reflect_coeff)
            
            # Calculate net driving pressure for SKK model
            net_driving = (feed_pressure - perm_pressure) - reflect_coeff * (feed_osm - perm_osm)
            
            print(f"\n  Stage {i}:")
            print(f"    Feed pressure: {feed_pressure:.2f} bar")
            print(f"    Feed osmotic pressure: {feed_osm:.2f} bar")
            print(f"    Permeate osmotic pressure: {perm_osm:.2f} bar")
            print(f"    Reflection coefficient (σ): {reflect_coeff:.3f}")
            print(f"    Net driving pressure: {net_driving:.2f} bar")
        
        # Calculate total power
        total_power_kw = sum(
            value(getattr(m.fs, f"pump{i}").work_mechanical[0]) / 1000
            for i in range(1, config_data['stage_count'] + 1)
        )
        print(f"\nTotal power consumption: {total_power_kw:.1f} kW")
    
    return results

In [6]:
# Build model with correct membrane property structure
def build_ro_model(config_data, feed_salinity_ppm, feed_temperature_c, membrane_type):
    """
    Build full WaterTAP RO model with calculated concentration polarization.
    """
    # Create concrete model
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Property package - using seawater
    m.fs.properties = props_sw.SeawaterParameterBlock()
    
    # Feed conditions
    feed_flow_m3_s = config_data['feed_flow_m3h'] / 3600  # Convert to m³/s
    feed_mass_frac = feed_salinity_ppm / 1e6  # Convert ppm to mass fraction
    
    # Create feed unit
    m.fs.feed = Feed(property_package=m.fs.properties)
    
    # Build stages
    n_stages = config_data['stage_count']
    
    # First, create all RO stages and pumps using setattr for proper parent assignment
    for i in range(1, n_stages + 1):
        # Create feed pump for stage using setattr
        setattr(m.fs, f"pump{i}", Pump(property_package=m.fs.properties))
        
        # Create RO stage with full configuration
        setattr(m.fs, f"ro_stage{i}", ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            concentration_polarization_type=ConcentrationPolarizationType.calculated,
            mass_transfer_coefficient=MassTransferCoefficient.calculated,
            pressure_change_type=PressureChangeType.fixed_per_stage
        ))
        
        # Create permeate product for each stage
        setattr(m.fs, f"stage_product{i}", Product(property_package=m.fs.properties))
    
    # Create final concentrate product
    m.fs.concentrate_product = Product(property_package=m.fs.properties)
    
    # Connect feed to first pump
    m.fs.feed_to_pump1 = Arc(
        source=m.fs.feed.outlet,
        destination=m.fs.pump1.inlet
    )
    
    # Connect first pump to first RO
    m.fs.pump1_to_ro1 = Arc(
        source=m.fs.pump1.outlet,
        destination=m.fs.ro_stage1.inlet
    )
    
    # Connect permeate from first RO to product
    m.fs.ro1_perm_to_prod = Arc(
        source=m.fs.ro_stage1.permeate,
        destination=m.fs.stage_product1.inlet
    )
    
    # Connect stages
    if n_stages > 1:
        for i in range(1, n_stages):
            # Connect concentrate of stage i to pump i+1
            setattr(
                m.fs, f"stage{i}_to_pump{i+1}",
                Arc(
                    source=getattr(m.fs, f"ro_stage{i}").retentate,
                    destination=getattr(m.fs, f"pump{i+1}").inlet
                )
            )
            # Connect pump i+1 to stage i+1
            setattr(
                m.fs, f"pump{i+1}_to_stage{i+1}",
                Arc(
                    source=getattr(m.fs, f"pump{i+1}").outlet,
                    destination=getattr(m.fs, f"ro_stage{i+1}").inlet
                )
            )
            # Connect permeate to product
            setattr(
                m.fs, f"ro{i+1}_perm_to_prod{i+1}",
                Arc(
                    source=getattr(m.fs, f"ro_stage{i+1}").permeate,
                    destination=getattr(m.fs, f"stage_product{i+1}").inlet
                )
            )
    
    # Connect final concentrate to product
    final_stage = n_stages
    m.fs.final_conc_arc = Arc(
        source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
        destination=m.fs.concentrate_product.inlet
    )
    
    # Apply arcs to expand the network
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    # NOW set membrane properties after model structure is built
    for i in range(1, n_stages + 1):
        stage_data = config_data['stages'][i-1]
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Default membrane properties based on type
        if membrane_type == "seawater":
            ro.A_comp.fix(1.5e-12)  # m/s/Pa
            ro.B_comp[0, 'TDS'].fix(1.0e-8)  # m/s
        else:  # brackish
            ro.A_comp.fix(4.2e-12)  # m/s/Pa
            ro.B_comp[0, 'TDS'].fix(3.5e-8)  # m/s
        
        # Set membrane area
        # Handle both 'membrane_area_m2' and 'area_m2' field names for compatibility
        if 'membrane_area_m2' in stage_data:
            required_area = stage_data['membrane_area_m2']
        elif 'area_m2' in stage_data:
            required_area = stage_data['area_m2']
        else:
            # Calculate from vessel count if not provided
            n_vessels = stage_data.get('n_vessels', stage_data.get('vessel_count', 1))
            vessel_area = 37.16 * 7  # Default: 37.16 m² per element * 7 elements per vessel
            required_area = n_vessels * vessel_area
        
        ro.area.fix(required_area)
        
        # Fixed pressure drop
        ro.deltaP.fix(-0.5e5 if i == 1 else -0.7e5)  # Pa
        ro.permeate.pressure[0].fix(101325)  # 1 atm
        
        # Channel properties
        ro.feed_side.channel_height.fix(0.001)  # 1 mm
        ro.feed_side.spacer_porosity.fix(0.85)
    
    # Set feed conditions
    m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(
        feed_flow_m3_s * 1000 * (1 - feed_mass_frac)
    )
    m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'].fix(
        feed_flow_m3_s * 1000 * feed_mass_frac
    )
    m.fs.feed.outlet.temperature.fix(273.15 + feed_temperature_c)
    m.fs.feed.outlet.pressure.fix(101325)  # 1 atm
    
    # Set pump efficiencies
    for i in range(1, n_stages + 1):
        getattr(m.fs, f"pump{i}").efficiency_pump.fix(0.8)
    
    # Set scaling
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
    calculate_scaling_factors(m)
    
    return m

In [7]:
def extract_results(m, config_data):
    """
    Extract simulation results from solved model.
    """
    results = {
        "status": "success",
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {}
    }
    
    # Overall performance
    total_feed_flow = value(m.fs.pump1.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                           m.fs.pump1.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
    
    total_permeate_flow = sum(
        value(getattr(m.fs, f"ro_stage{i}").permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
              getattr(m.fs, f"ro_stage{i}").permeate.flow_mass_phase_comp[0, 'Liq', 'TDS'])
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    results["performance"]["total_recovery"] = total_permeate_flow / total_feed_flow
    
    # Energy consumption
    total_power_kw = sum(
        value(getattr(m.fs, f"pump{i}").work_mechanical[0]) / 1000  # Convert W to kW
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    results["economics"]["total_power_kw"] = total_power_kw
    results["economics"]["specific_energy_kwh_m3"] = (
        total_power_kw / (total_permeate_flow * 3.6)  # m�/s to m�/h
    )
    
    # Stage results
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        pump = getattr(m.fs, f"pump{i}")
        
        stage_result = {
            "stage_number": i,
            "feed_pressure_bar": value(ro.inlet.pressure[0]) / 1e5,
            "permeate_flow_m3h": value(
                ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                ro.permeate.flow_mass_phase_comp[0, 'Liq', 'TDS']
            ) * 3.6,  # kg/s to m�/h
            "permeate_tds_ppm": value(
                ro.permeate.flow_mass_phase_comp[0, 'Liq', 'TDS'] /
                (ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                 ro.permeate.flow_mass_phase_comp[0, 'Liq', 'TDS'])
            ) * 1e6,
            "concentrate_flow_m3h": value(
                ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                ro.retentate.flow_mass_phase_comp[0, 'Liq', 'TDS']
            ) * 3.6,
            "pump_power_kw": value(pump.work_mechanical[0]) / 1000
        }
        results["stage_results"].append(stage_result)
    
    # Mass balance verification
    results["mass_balance"]["feed_flow_m3h"] = total_feed_flow * 3.6
    results["mass_balance"]["total_permeate_m3h"] = total_permeate_flow * 3.6
    
    final_stage = config_data['stage_count']
    final_ro = getattr(m.fs, f"ro_stage{final_stage}")
    results["mass_balance"]["final_concentrate_m3h"] = value(
        final_ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
        final_ro.retentate.flow_mass_phase_comp[0, 'Liq', 'TDS']
    ) * 3.6
    
    return results

## Run Simulation

In [8]:
# Build model
try:
    print("Building simplified RO model...")
    m = build_ro_model_simple(configuration, feed_salinity_ppm, feed_temperature_c, membrane_type)
    
    print("\nInitializing and solving model...")
    solve_results = initialize_and_solve_elegant(m, configuration, optimize_pumps)
    
    if solve_results.solver.termination_condition == TerminationCondition.optimal:
        print("\nExtracting results...")
        results = extract_results(m, configuration)
        results["message"] = "Simulation completed successfully"
    else:
        results = {
            "status": "error",
            "message": f"Solver failed: {solve_results.solver.termination_condition}",
            "performance": {},
            "economics": {},
            "stage_results": [],
            "mass_balance": {}
        }
        
except Exception as e:
    results = {
        "status": "error",
        "message": f"Simulation error: {str(e)}",
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {}
    }

print("\nSimulation complete.")

Building simplified RO model...
2025-07-15 00:41:03 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.pump1.control_volume.work


2025-07-15 00:41:03 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.ro_stage1.area


2025-07-15 00:41:03 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.pump2.control_volume.work


2025-07-15 00:41:03 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.ro_stage2.area



Initializing and solving model...

Simulation complete.


## Display Results

In [9]:
# Display results summary
import json
print("\n" + "="*50)
print("SIMULATION RESULTS")
print("="*50)
print(json.dumps(results, indent=2))


SIMULATION RESULTS
{
  "status": "error",
  "message": "Simulation error: name 'initialize_and_solve_elegant' is not defined",
  "performance": {},
  "economics": {},
  "stage_results": [],
  "mass_balance": {}
}


In [10]:
# Results cell - tagged for papermill to extract
results

{'status': 'error',
 'message': "Simulation error: name 'initialize_and_solve_elegant' is not defined",
 'performance': {},
 'economics': {},
 'stage_results': [],
 'mass_balance': {}}